# INFERENCE

In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import cv2

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Flatten, Rescaling, Input
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image

#from plot_keras_history import show_history, plot_history

In [ ]:
# Le dossier des images
DATA = "/kaggle/input/archive-21/Archive_21"

In [ ]:
# Le modèle (example : /kaggle/input/model_missing_stud_lh07_v6/keras/default/1/model_missing_stud_LH07.keras)
model = load_model("/kaggle/input/model_missing_stud_lh07_v6/keras/default/1/model_missing_stud_LH07.keras")

In [ ]:
# Choix défaut

#NOM_DEFAUT = "LH03_missing_tape"
#NOM_DEFAUT = "LH03_tape"
#NOM_DEFAUT = "LH03_double_tape"
#NOM_DEFAUT = "LH04_double_tape"
#NOM_DEFAUT = "LH04_short_tape"
#NOM_DEFAUT = "LH05_tilted_tape"
#NOM_DEFAUT = "RH07_missing_tape"
NOM_DEFAUT = "LH07_missing_stud"

In [ ]:
match = {"LH03_missing_tape":["LH",
                              "_03_",
                              "model_missing_tape_LH03",0.09,0.18,0.05,0.25],
        "LH03_double_tape":["LH",
                              "_03_",
                              "model_double_tape_LH03",
                              0.07,0.15,0.05,0.15],
         "LH03_tape":["LH",
                              "_03_",
                              "model_tape_LH03",
                              0.09,0.18,0.05,0.25],
        "LH04_double_tape":["LH",
                            "_04_",
                            "model_double_tape_LH04",
                            0.28,0.35,0,0.07],
        "LH04_short_tape":["LH",
                            "_04_",
                            "model_short_tape_LH04",
                           0.3,0.4,0,0.22],
        "LH05_tilted_tape":["LH",
                            "_05_",
                            "model_tilted_tape_LH05",
                            0.4,0.49,0.78,1],
        "RH07_missing_tape":["RH",
                            "_07_",
                            "model_missing_tape_RH07",
                             0.62,0.73,0.78,0.91],
        "LH07_missing_stud":["LH",
                            "_07_",
                            "model_missing_stud_LH07",0.57,0.6,0.31,0.37]}

In [ ]:
PREFIX = match[NOM_DEFAUT][0]
SUFFIX = match[NOM_DEFAUT][1]
MODEL = match[NOM_DEFAUT][2]
COEF_X1 = match[NOM_DEFAUT][3]
COEF_X2 = match[NOM_DEFAUT][4]
COEF_Y1 = match[NOM_DEFAUT][5]
COEF_Y2 = match[NOM_DEFAUT][6]

In [ ]:
shutil.rmtree("/kaggle/working/",ignore_errors=True)

# Préparation des images
on va extraire un morceau des images et on va appliquer le modèle sur ces morceaux.

In [ ]:
img = plt.imread(glob.glob(f"{DATA}/*/{PREFIX}*{SUFFIX}*")[0])
taille_image = img.shape
taille_image

In [ ]:
plt.imshow(img)

In [ ]:
# Visualisation d'un morceau sur image
img = plt.imread(glob.glob(f"{DATA}/*/{PREFIX}*{SUFFIX}*")[0])
y_start, y_end = int(taille_image[0]*COEF_Y1), int(taille_image[0]*COEF_Y2)
x_start, x_end = int(taille_image[1]*COEF_X1), int(taille_image[1]*COEF_X2)
img_cropped = img[y_start:y_end, x_start:x_end]
img_cropped = cv2.resize(img_cropped, (224, 224))
plt.imshow(img_cropped)

## Création de nouveaux dossiers avec les images préparées

In [ ]:
# Crée un dossier dir_crop à partir du dossier dir avec des sous dossiers "C" et "NC" qui reprend les images de dir 
# en les réduisant
def preparation_image(dir_source, dir_crop):
    
    for img in glob.glob(os.path.join(dir_source, '**', f"{PREFIX}*{SUFFIX}*.bmp"), recursive=True):
        # Load the image
        image_path = img 
        image = cv2.imread(image_path)

        # Crop the window
        cropped_image = image[y_start:y_end, x_start:x_end]

        # Créer le dossier s'il n'existe pas
        dossier = dir_crop #+ r"/C"
        os.makedirs(dossier, exist_ok=True)

        # Chemin d'accès pour sauvegarder l'image préparée
        cropped_image_path = os.path.join(dossier, os.path.basename(img))

        # Sauvegarder l'image au même nom que l'image d'origine
        cv2.imwrite(cropped_image_path, cropped_image)

In [ ]:
# Définir le chemins vers le répertoire avec les morceaux d'image
DATA_CROP = "/kaggle/working/data/DATA_CROP"

os.makedirs("/kaggle/working/data/DATA_CROP", exist_ok=True)

In [ ]:
# Préparation des images
preparation_image(DATA, DATA_CROP)

In [ ]:
%%time
# Visualisons quelques images
fig, ax = plt.subplots(6//3, 3, figsize=(30,15))
ax = ax.flatten()
for idx, path in enumerate(glob.glob(DATA+f"/*/{PREFIX}*{SUFFIX}*")[0:6]):
    img = os.path.join(DATA_CROP, path)
    ax[idx].imshow(plt.imread(img))
    ax[idx].set_title(path, fontsize=20)

# INFERENCE

In [ ]:
# Créer un générateur d'images pour la prédiction
predict_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_input
)

# Spécifiez le répertoire contenant les images pour la prédiction
PREDICT_DIR = "/kaggle/working/data"

predict_generator = predict_datagen.flow_from_directory(
    PREDICT_DIR,
    target_size=(224, 224),
    batch_size=1,
    class_mode=None,
    shuffle=False
)

# Faire les prédictions
predictions = model.predict(predict_generator, steps=len(predict_generator), verbose=1)

# Récupérer les noms de fichiers pour lier les prédictions aux images
filenames = predict_generator.filenames

In [ ]:
# Associer chaque fichier d'image à sa prédiction
liste_defaut = []
for filename, prediction in zip(filenames, predictions):
    predicted_class = np.argmax(prediction)
    if predicted_class == 1:
        print(f"Image: {filename} - Prediction: {predicted_class}")
        liste_defaut.append(filename)

In [ ]:
# Visualisons les images de défaut détectés
fig, ax = plt.subplots(1+len(liste_defaut)//4, 4, figsize=(10, 10))
for idx, img in enumerate(liste_defaut) :
    ax = ax.flatten()
    ax[idx].imshow(plt.imread(glob.glob(DATA+f"/*/{os.path.basename(img)}")[0]))
    ax[idx].set_title(os.path.basename(img), fontsize=7)

In [ ]:
# Exporter la liste des images NC
export = pd.DataFrame({NOM_DEFAUT:liste_defaut})
export.to_csv("resultat.csv", index=False)